Import the libraries

In [1]:
import json
import sqlite3
import pandas as pd
from langchain_openai import ChatOpenAI
import os
import re
import chromadb
from openai import OpenAI
from langchain_community.callbacks import get_openai_callback

Connect to sqlite db

In [2]:
cols = ['showname','first_airing','imdb','lang','description']
def fetch_rows(limit: int, offset: int):
    conn = sqlite3.connect("movie_db.sqlite")
    cursor = conn.cursor()
    executor = cursor.execute(f'select {",".join(cols)} from tvmaze where metadata is null and description is not null order by showname asc limit {limit} offset {offset}')
    data = executor.fetchall()
    cursor.close()
    conn.close()
    return data

pd.DataFrame(fetch_rows(100, 0), columns=cols)

,showname,first_airing,imdb,lang,description
0,'Allo 'Allo!,1982-12-30,tt0086659,English,"<p>In this spoof of World War II, René Artois ..."
1,'Orrible,2001-09-10,tt0299233,English,<p>Paul Clark is a cab driver and wannabe smal...
2,'Run Sbit,2016-04-01,None,Welsh,<p>Satirical comedy series in a fly on the wal...
3,'Til Death Do Us Part,2007-03-19,None,English,<p><b>'Til Death Do Us Part</b> is murder-myst...
4,'Til Death Do Us Part,2019-07-09,tt10553838,English,<p><b>'Til Death Do Us Part</b> follows lovers...
...,...,...,...,...,...
95,A Real Bug's Life,None,None,English,<p><b>A Real Bug's Life</b> is an adventure in...
96,A Sister's All You Need.,2017-10-08,tt9731082,Japanese,<p>This is the story about the daily life of a...
97,A Small Light,2023-05-01,tt17921714,English,<p><b>A Small Light</b> follows twenty-somethi...
98,A Soldier's Heart,2020-01-20,tt12132812,Tagalog,<p>The story of seven individuals who chose th...


Prepare Labeling Agent

In [3]:
# Initialize a ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv('OPENAI_KEY'),
    # model="deepseek-r1-distill-llama-8b",
    # openai_api_base='http://127.0.0.1:1234/v1',
)

Tools for extracting metadata

In [4]:
def generate_prompt(values: list[str]):
    with open(f'./prompts/movie_metadata_extraction.txt', 'r') as file:
        data = file.read().rstrip()
        index=0
            
        while(data.find(f"!<INPUT_{index}>") > 0):
            data = data.replace(f"!<INPUT_{index}>", values[index] if len(values) > index is not None else '')
            index+=1
        return data

    return None;

Define job to process data

In [6]:
from langchain.schema import HumanMessage
from IPython.display import clear_output

def process_job(limit: int, offset: int) -> list[dict]:
    results = []
    completed = 0
    data = fetch_rows(limit, offset)
    total_tokens = 0
    total_cached_tokens = 0
    total_costs = 0
    for row in data:
        if(row[4] is not None and len(row[4]) > 0):
            with get_openai_callback() as cb:  
                prompt = generate_prompt([row[4]])
                output = llm.invoke([HumanMessage(content=prompt)])
                total_tokens+=cb.total_tokens
                total_costs+=cb.total_cost
                total_cached_tokens+=cb.prompt_tokens_cached
                json_str = re.findall(r"```json(.*?)```", output.text(),re.DOTALL)
                if(len(json_str) > 0):
                    try:
                        structured_data = json.loads(json_str[0])
                        conn = sqlite3.connect("movie_db.sqlite")
                        cursor = conn.cursor()
                        showname:str = row[0].replace("'", "\\'")
                        cursor.execute(f'update tvmaze set metadata = \'{json.dumps(structured_data)}\' where showname = \'{showname}\'')
                        conn.commit()
                        cursor.close()
                        conn.close()
                        results.append(structured_data)
                    except Exception  as e:
                        # print(row[0])
                        # print(e)
                        pass
                completed+=1
                clear_output(wait=True)
                print(f"{completed}/{limit} done ({round(completed/limit, 2) * 100}%)")
                print(f"total token use: {total_tokens}")
                print(f"total cost: ${total_costs}")
                print('-------------')
                print(json_str)
            
    return results;

Process the data

In [7]:
results = process_job(10000,0)

9843/10000 done (98.0%)
total token use: 2483823
total cost: $0.655994249999999
-------------
['\n{\n  "locations": ["Las Vegas", "Beverly Hills"],\n  "characters": ["former Las Vegas showgirl", "blue-blooded in-laws"],\n  "time period": [],\n  "events": ["inheritance"],\n  "sentimental": ["widow"],\n  "genre": ["comedy"]\n}\n']


Define tools for vector embeddings

In [ ]:
client = OpenAI(api_key="fake", base_url='http://127.0.0.1:1234/v1')
chroma_client = chromadb.PersistentClient(path="./chroma_db")  
collection = chroma_client.get_or_create_collection(name="movies")

def get_embedding(text):
    response = client.embeddings.create(model="text-embedding-nomic-embed-text-v1.5", input=text)
    return response.data[0].embedding

def db_to_embeddings(limit: int, offset: int):
    cols = ['showname','first_airing','imdb','lang','description', 'metadata']
    conn = sqlite3.connect("movie_db.sqlite")
    cursor = conn.cursor()
    executor = cursor.execute(f'select {",".join(cols)} from tvmaze where metadata is not null and processed is null order by showname asc limit {limit} offset {offset}')
    rows = executor.fetchall()

    
    for row in rows:
        title = row[0]
        description=  row[4]
        id = f"{title}-{row[2]}"
        text = f"{title} {description}" 
        language = row[3] 

        embedding = get_embedding(text)
        metadata : dict = json.loads(row[5])
        parsed_metadata: dict = {}
        
        for key in metadata.keys():
            value = metadata[key]
            if(len(value) == 0):
                continue
            if(type(metadata[key]) is list):
                parsed_metadata[key] = ','.join(value)
            if(type(metadata[key]) is int or type(value) is str):
                parsed_metadata[key] = value

        parsed_metadata["title"] = title if title is not None else ''
        parsed_metadata["language"] = language if language is not None else ''
        parsed_metadata["aired"] = row[1] if row[1] is not None else ''
        print(parsed_metadata)
      
        collection.add(
            ids=[id],
            embeddings=[embedding],
            metadatas=[parsed_metadata]
        )
        
        cursor = conn.cursor()
        cursor.execute(f'update tvmaze set processed = true where showname = \'{title}\'')
        conn.commit()
    cursor.close()
    conn.close()
      

Get word embeddings

In [ ]:
db_to_embeddings(10,0)

Add of existing embedding ID: 3B no Koibito-None
Insert of existing embedding ID: 3B no Koibito-None


{'characters': 'Kobayashi Haruka,Shintaro,Yu,Yoshi', 'genre': 'romance,drama', 'title': '3B no Koibito', 'language': 'Japanese', 'aired': '2021-01-09'}


OperationalError: database is locked

Testing query

In [75]:
query_text = "movies about history"
query_embedding = get_embedding(query_text)

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5
)

for movie in results["metadatas"][0]:
    print(f"Title: {movie['title']} | Language: {movie['language']}")

Title: 1000 Years of History | Language: English
Title: 12 Monkeys | Language: English
Title: 14 - Diaries of the Great War | Language: French
Title: 1066: A Year to Conquer England | Language: English
Title: 11.22.63 | Language: English
